# Test LOF and OC-SVM

In [2]:
import pandas as pd
import numpy as np
import os
import json
import torch
import torch.nn as nn
from config import whisper_config
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from model import AutoEncoder, Dataset, train_kmeans, test_kmeans, train_ae, test_ae, get_flows, transform
from model import get_metrics
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

MAX_LEN = whisper_config["n_fft"] * 2

/root/anaconda3/envs/PRO/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
benign_filenames = [os.path.join("train_set", "benign" + str(i) + ".csv") 
                    for i in range(1, 3)]
attack_filenames = [os.path.join("attack_set", x) for x in 
                    os.listdir("attack_set") if x.endswith(".csv")]

In [4]:
train_benign_filename = "dataset/benign_small.csv"

train_df = pd.read_csv(train_benign_filename)
train_df["label"] = 1
train_packet_data, train_packet_labels, train_flow_data, train_flow_labels \
= transform(get_flows(train_df))
train_data =  train_flow_data + train_packet_data
train_labels = train_flow_labels + train_packet_labels

/root/anaconda3/envs/PRO/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [5]:
lof = LocalOutlierFactor(novelty=True)
lof.fit(train_data)

LocalOutlierFactor(novelty=True)

In [6]:
ocs = OneClassSVM(kernel="rbf", nu=0.1)
ocs.fit(train_data)

OneClassSVM(nu=0.1)

In [7]:
with open("attacker-ips.json", "r") as f:
    attack_ips_dict = json.load(f)

accuracy_dict = {
    "lof": {},
    "ocs": {}
}

for filename in attack_filenames:
    benign_df = pd.read_csv(benign_filenames[0])
    attack_df = pd.read_csv(filename)
    test_df = pd.concat([benign_df, attack_df], ignore_index=True)
    # test_df = pd.read_csv(filename)
    
    file_key = os.path.basename(filename).split(".")[0]
    cur_attack_ips = attack_ips_dict.get(file_key, [])
    test_df["label"] = 0
    for row in test_df.iterrows():
        if row[1]["src_ip"] in cur_attack_ips or row[1]["dst_ip"] in cur_attack_ips:
            test_df.loc[row[0], "label"] = -1
        else:
            test_df.loc[row[0], "label"] = 1

    test_packet_data, test_packet_labels, test_flow_data, test_flow_labels \
    = transform(get_flows(test_df), test_data_aug=False)
    test_data =  test_flow_data + test_packet_data
    test_labels = test_flow_labels + test_packet_labels

    lof_pred = lof.predict(test_data)
    lof_metrics = get_metrics(test_labels, lof_pred)
    print(f"lof metrics of {filename}: {lof_metrics}")
    accuracy_dict["lof"][filename] = lof_metrics

    ocs_pred = ocs.predict(test_data)
    ocs_metrics = get_metrics(test_labels, ocs_pred)
    print(f"ocs metrics of {filename}: {ocs_metrics}")
    accuracy_dict["ocs"][filename] = ocs_metrics

metrics_save_dir = os.path.join("result", "unsupervised",
                    os.path.basename(train_benign_filename))
accuracy_save_path = os.path.join(metrics_save_dir, "all-metrics.json")
os.makedirs(os.path.dirname(accuracy_save_path), exist_ok=True)
with open(accuracy_save_path, "w") as f:
    json.dump(accuracy_dict, f, indent=4)

lof metrics of attack_set/LDoS_small.csv: (0.749535297607277, 1.0, 0.7279639175257732, 0.8425684241927064, nan, nan, 0.3797673302950348, 0.24046533940993045)


KeyError: LocalOutlierFactor(novelty=True)